# 9. A Pythonic Object

## 9.1 @classmethod and @staticmethod

@classmethod 데커레이터는 객체가 아닌 클래스에 연산을 수행하는 메서드를 정의하는 데커레이터다.

In [1]:
class Demo :
    @classmethod
    def klassmeth(*args) :
        return args
    @staticmethod
    def statmeth(*args) :
        return args

In [3]:
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')


@classmethod는 메서드가 호출되는 방식을 변경해서 클래스 자체를 첫 번째 인수로 받게 만들며, 대안 생성자를 구현하기 위해 주로 사용된다. 관습적으로 cls를 클래스 메서드의 첫 번째 매개변수명으로 사용하지만, 파이썬은 특정 매개변수명을 요구하지는 않는다.

실제로 위의 코드에서 호출 방법에 무관하게 Demo 클래스를 첫 번째 인수로 받는 것을 확인할 수 있다!

In [4]:
print(Demo.statmeth('spam'))

('spam',)


반대로 @staticmethod 데커레이터는 메서드가 특별한 첫 번째 인수를 받지 않도록 메서드를 변경한다. 본질적으로 정적 메서드는 모듈 대신 클래스 본체 안에 정의된 평범한 함수일 뿐이다.

(참고로 책의 저자는 @staticmethod를 거의 사용하지 않는 듯하다.)

## 9.2 비공개 속성 및 보호된 속성

파이썬은 private 키워드가 있는 자바와 달리 비공개 변수를 생성할 수 있는 방법이 **없다.** 다만 서브클래스에서 비공개 성격의 속성을 실수로 변경하지 못하게 하는 방법은 있다.

그것은 속성명을 \_\_method처럼 두 개의 언더바(\_)로 시작하며 언더바 없이 혹은 하나의 언더바로 끝나도록 정의하면 파이썬은 자체적으로 언더바와 클래스명을 변수명 앞에 붙여 객체의 \_\_dict\_\_에 저장한다. 이를 **이름 장식** 기능이라고 부른다.

하지만 이는 고의적인 접근은 막지 못하기에 완전한 해결책은 아니다.

파이썬 개발자들은 **관례적으로** 언더바 하나를 붙여 보호된 속성을 나타낸다. (이 또한 강제적 사항은 아니다.)

## 9.3 \_\_slot\_\_ 클래스

\_\_slot\_\_ 클래스 속성을 이용하여 메모리 사용량을 획기적으로 줄일 수 있다. \_\_slot\_\_ 속성은 파이썬 인터프리터가 객체 속성을 딕셔너리 대신 튜플에 저장하게 만든다.

\_\_slot\_\_을 클래스에 정의함으로써 '이 속성들이 이 클래스 객체가 가지는 속성'임을 인터프리터에 알려준다. 그러면 파이썬 인터프리터는 이 속성들을 각 객체의 튜플형 구조체에 저장하여 \_\_dict\_\_ 속성을 각 객체마다 유지하는 부담을 덜어낸다.

# 10. Sequence Hacking, Hashing, and Slicing

## 10.1 프로토콜과 덕 타이핑

객체지향 프로그래밍에서, 프로토콜은 문서에만 정의되어 있고 실제 코드에서는 정의되지 않는 비공식 인터페이스다.

표준 시그너처와 의미에 따라 이 메서드들을 구현하는 어떠한 클래스도 시퀀스가 필요한 곳에 사용될 수 있다. 그 클래스의 슈퍼클래스가 무엇인지는 중요치 않다. 단지 필요한 메서드만 제공하면 된다.

아래에서 시퀀스 프로토콜을 메서드로 구현하는 예시를 살펴 보자.

## 10.2 \_\_getitem\_\_()

시퀀스 프로토콜의 첫 번째 예시로, 슬라이싱이 어떻게 작동하는지 보도록 하자.

In [5]:
class MySeq :
    def __getitem__(self, index) :
        return index

s = MySeq()
print(s[1])
print(s[1:4])
print(s[1:4:2])
print(s[1:4:2, 9])
print(s[1:4:2, 7:9])

1
slice(1, 4, None)
slice(1, 4, 2)
(slice(1, 4, 2), 9)
(slice(1, 4, 2), slice(7, 9, None))


위 코드에서 보듯, 슬라이싱을 하게 되면 slicce 자료형이 생성되어 \_\_getitem\_\_()이 이를 인자로 받게 된다. 세 인자 중 없는 인자는 기본값인 None으로 대입되는 걸 볼 수 있다. 한편 [] 안에 콤마가 들어갈 경우 \_\_getitem\_\_()이 튜플의 형태로 받는 것을 알 수 있다.

## 10.3 \_\_getattr\_\_()

두 번째 예시는 **동적 속성 접근**이다. 이는 \_\_getattr\_\_() 메서드를 구현하여 실현할 수 있다.

파이썬 인터프리터는 해당 이름의 속성을 찾지 못하면 \_\_getattr\_\_() 메서드를 호출한다. 이 과정을 간단히 말하면, 우선 파이썬은 사용자 정의 객체에 해당 속성이 있는지 검사한다. 속성이 없으면 이 객체의 클래스에서 더 검색한다. 이후 상속 그래프를 따라 계속 올라간다. 그래도 해당 속성을 찾지 못하면 self와 속성명을 문자열로 전달해서 \_\_getattr\_\_() 메서드를 호출한다.

## 10.4 그 외

책에서는 위의 메서드들 외에도 Vector 클래스에 여러 메서드(\_\_hash\_\_(), \_\_format\_\_() 등)를 추가하는 것을 코드로 보여주었다. 이는 책에 적혀 있으니 굳이 여기에 옮기지는 않는다.